In [1]:
# 필요한 라이브러리 임포트
from binance.client import Client
from binance.enums import *
import pandas as pd
from datetime import datetime , timedelta
import time

import pandas as pd
import numpy as np
import torch
import pyarrow

In [2]:
# API 키 설정
api_key = ''
api_secret = ''

# 바이낸스 클라이언트 초기화
client = Client(api_key, api_secret)

In [3]:
class Future_db:
    def __init__(self):
        self.futures = None
        self.curent_symbols = self.get_all_futures_symbols()
        self.top10_futures = None
    
    def get_futures_klines(self, symbol='BTCUSDT', interval='30m', limit=500):
        """
        선물 캔들스틱 데이터 가져오기
        :param symbol: 거래쌍 (예: 'BTCUSDT')
        :param interval: 시간간격 (1m, 3m, 5m, 15m, 30m, 1h, 2h, 4h, 6h, 8h, 12h, 1d, 3d, 1w, 1M)
        :param limit: 가져올 캔들 수
        :return: DataFrame
        """
        try:
            klines = client.futures_klines(symbol=symbol, interval=interval, limit=limit)
            
            # DataFrame 생성
            df = pd.DataFrame(klines, columns=['timestamp', 'open', 'high', 'low', 'close', 
                                            'volume', 'close_time', 'quote_volume', 'trades',
                                            'taker_base', 'taker_quote', 'ignore'])
            
            # 데이터 타입 변환
            df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
            for col in ['open', 'high', 'low', 'close', 'volume', 'quote_volume']:
                df[col] = df[col].astype(float)
                
            return df[['timestamp', 'open', 'high', 'low', 'close', 'volume', 'quote_volume']]
        
        except Exception as e:
            print(f"에러 발생: {e}")
            return None

    def get_futures_position(self, symbol='BTCUSDT'):
        """
        현재 선물 포지션 정보 가져오기
        :param symbol: 거래쌍
        :return: Dictionary
        """
        try:
            position = client.futures_position_information(symbol=symbol)
            return position
        except Exception as e:
            print(f"에러 발생: {e}")
            return None

    def get_all_futures_symbols(self):
        """
        거래 가능한 모든 선물 심볼 목록 반환
        """
        try:
            exchange_info = client.futures_exchange_info()
            return [symbol['symbol'] for symbol in exchange_info['symbols'] if symbol['status'] == 'TRADING']
        except Exception as e:
            print(f"심볼 목록 조회 에러: {e}")
            return None

    def get_futures_historical_klines(self, symbol='BTCUSDT', interval='30m', start_str='2017-01-01', end_str='2025-04-09'):
        """
        특정 기간의 선물 캔들스틱 데이터 가져오기
        :param symbol: 거래쌍 또는 'all'
        :param interval: 시간간격
        :param start_str: 시작 날짜 (YYYY-MM-DD)
        :param end_str: 종료 날짜 (YYYY-MM-DD)
        :return: Dictionary of DataFrames (symbol을 'all'로 지정한 경우) 또는 단일 DataFrame
        """
        try:
            # 날짜를 타임스탬프로 변환
            start_ts = int(pd.Timestamp(start_str).timestamp() * 1000)
            end_ts = int(pd.Timestamp(end_str).timestamp() * 1000)

            if symbol.lower() == 'all':
                symbols = get_all_futures_symbols()
                if not symbols:
                    return None
                
                result = {}
                for sym in symbols:
                    print(f"{sym} 데이터 수집 중...")
                    klines = []
                    while start_ts < end_ts:
                        temp_data = client.futures_historical_klines(
                            symbol=sym,
                            interval=interval,
                            start_str=str(start_ts),
                            end_str=str(end_ts),
                            limit=1000
                        )
                        if not temp_data:
                            break
                        klines.extend(temp_data)
                        start_ts = temp_data[-1][0] + 1  # 다음 시작점을 마지막 데이터의 다음 시점으로 설정
                        time.sleep(0.1)  # API 호출 제한 방지
                    
                    if klines:
                        df = pd.DataFrame(klines, columns=['timestamp', 'open', 'high', 'low', 'close', 
                                                        'volume', 'close_time', 'quote_volume', 'trades',
                                                        'taker_base', 'taker_quote', 'ignore'])
                        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
                        for col in ['open', 'high', 'low', 'close', 'volume', 'quote_volume']:
                            df[col] = df[col].astype(float)
                        result[sym] = df[['timestamp', 'open', 'high', 'low', 'close', 'volume', 'quote_volume']]
                    time.sleep(0.5)
                return result
            
            else:
                klines = []
                while start_ts < end_ts:
                    temp_data = client.futures_historical_klines(
                        symbol=symbol,
                        interval=interval,
                        start_str=str(start_ts),
                        end_str=str(end_ts),
                        limit=1000
                    )
                    if not temp_data:
                        break
                    klines.extend(temp_data)
                    start_ts = temp_data[-1][0] + 1  # 다음 시작점을 마지막 데이터의 다음 시점으로 설정
                    time.sleep(0.1)  # API 호출 제한 방지
                
                df = pd.DataFrame(klines, columns=['timestamp', 'open', 'high', 'low', 'close', 
                                                'volume', 'close_time', 'quote_volume', 'trades',
                                                'taker_base', 'taker_quote', 'ignore'])
                
                df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
                for col in ['open', 'high', 'low', 'close', 'volume', 'quote_volume']:
                    df[col] = df[col].astype(float)
                    
                return df[['timestamp', 'open', 'high', 'low', 'close', 'volume']]
        
        except Exception as e:
            print(f"에러 발생: {e}")
            return None
    def get_market_cap_rank(self, start_date = None, end_date = None, ranges = None):
        """
        start_date : 시작 날짜 
        end_date = 종료 날짜 
        ranges = 심볼 범위 지정
        """
        if self.curent_symbols is None:
            symbols = get_all_futures_symbols()
        else:
            symbols = self.curent_symbols
        
        if ranges is not None:
            symbols = symbols[:ranges]

        # 기준: 전날 날짜 기준 한달 전부터
        if start_date is None and end_date is None:
            today = datetime.now().date()
            end_date = today - timedelta(days=1)
            start_date = end_date - timedelta(days=30)
        else:
            start_date = datetime.strptime(start_date, "%Y-%m-%d")
            end_date  = datetime.strptime(end_date, "%Y-%m-%d")
        print(f"데이터 수집 기간: {start_date} ~ {end_date}")

        results = []

        for i, symbol in enumerate(symbols):
            print(f"[{i+1}/{len(symbols)}] {symbol} 처리 중...")

            try:
                df = self.get_futures_historical_klines(
                    symbol=symbol,
                    interval='1d',
                    start_str=start_date.strftime('%Y-%m-%d'),
                    end_str=end_date.strftime('%Y-%m-%d')
                )

                if df is None or df.empty:
                    continue

                avg_price = df['close'].mean()
                avg_volume = df['volume'].mean()
                market_cap = avg_price * avg_volume

                results.append({
                    'symbol': symbol,
                    'avg_price': avg_price,
                    'avg_volume': avg_volume,
                    'market_cap': market_cap
                })

                time.sleep(0.2)  # API 제한 방지
            except Exception as e:
                print(f"{symbol} 처리 중 에러 발생: {e}")
                continue

        # 정렬
        df_result = pd.DataFrame(results)
        df_result = df_result.sort_values(by='market_cap', ascending=False).reset_index(drop=True)

        return df_result

    def get_top10_futures(self, interval = '30m',symbols_option=1, symobls_list = None):
        """시가총액 기준 상위 10개 선물 종목을 반환합니다.
        
        (self, symbols_option=1, symobls_list == None)

        Param

        interval = '30m'  : 시간간격 (1m, 3m, 5m, 15m, 30m, 1h, 2h, 4h, 6h, 8h, 12h, 1d, 3d, 1w, 1M)
        symbols_option==1 : api에서 받은 심볼을 슬라이싱 합니다.
        symbols_option==2 : 현재로부터 한달전의 마켓 켑을 두고, 약 422개를 계산한 후 10를 선별합니다.
        symbols_option==3 : 임의로 지정한 symbols_list 를 심볼로 사용합니다.(날짜를 변경할 때 사용)
            symobls_list: 심볼리스트를 지정해서 데이터를 가져옵니다.(option 3와 함꼐사용)
        """
        if symbols_option == 1:
            symbols = self.curent_symbols[:10]
        elif symbols_option ==2:
            symbols_marketcap = self.get_market_cap_rank()
            symbols = list(symbols_marketcap['symbol'][:10])
        elif symbols_option == 3:
            symbols = symobls_list
        dfs = []

        for i, symbol in enumerate(symbols):
            print(f"[{i+1}/{len(symbols)}] {symbol} 처리 중...")
            tmp = self.get_futures_historical_klines(
                symbol=symbol,
                interval='30m',
                end_str=datetime.now().strftime('%Y-%m-%d')
            )

            if tmp is None or tmp.empty:
                continue

            # 'timestamp'를 인덱스로 하고, 'close'만 추출해서 컬럼명 바꾸기
            close_series = tmp.set_index('timestamp')['close'].rename(symbol)
            dfs.append(close_series)

        merged_df = pd.concat(dfs, axis=1)
        merged_df = merged_df.sort_index().reset_index()  # timestamp 복원
        merged_df.set_index('timestamp', inplace=True)
        return merged_df
    

In [4]:
#class 객체 생성
futures = Future_db()

In [5]:
#심볼 확인
symbols_marketcap = futures.get_market_cap_rank(start_date='2024-03-08', end_date='2024-04-07')
symbols_marketcap

데이터 수집 기간: 2024-03-08 00:00:00 ~ 2024-04-07 00:00:00
[1/423] BTCUSDT 처리 중...
[2/423] ETHUSDT 처리 중...
[3/423] BCHUSDT 처리 중...
[4/423] XRPUSDT 처리 중...
[5/423] EOSUSDT 처리 중...
[6/423] LTCUSDT 처리 중...
[7/423] TRXUSDT 처리 중...
[8/423] ETCUSDT 처리 중...
[9/423] LINKUSDT 처리 중...
[10/423] XLMUSDT 처리 중...
[11/423] ADAUSDT 처리 중...
[12/423] XMRUSDT 처리 중...
[13/423] DASHUSDT 처리 중...
[14/423] ZECUSDT 처리 중...
[15/423] XTZUSDT 처리 중...
[16/423] BNBUSDT 처리 중...
[17/423] ATOMUSDT 처리 중...
[18/423] ONTUSDT 처리 중...
[19/423] IOTAUSDT 처리 중...
[20/423] BATUSDT 처리 중...
[21/423] VETUSDT 처리 중...
[22/423] NEOUSDT 처리 중...
[23/423] QTUMUSDT 처리 중...
[24/423] IOSTUSDT 처리 중...
[25/423] THETAUSDT 처리 중...
[26/423] ALGOUSDT 처리 중...
[27/423] ZILUSDT 처리 중...
[28/423] KNCUSDT 처리 중...
[29/423] ZRXUSDT 처리 중...
[30/423] COMPUSDT 처리 중...
[31/423] DOGEUSDT 처리 중...
[32/423] SXPUSDT 처리 중...
[33/423] KAVAUSDT 처리 중...
[34/423] BANDUSDT 처리 중...
[35/423] RLCUSDT 처리 중...
[36/423] MKRUSDT 처리 중...
[37/423] SNXUSDT 처리 중...
[38/423] DOTUSDT 처

,symbol,avg_price,avg_volume,market_cap
0,BTCUSDT,68461.051613,3.399354e+05,2.327234e+10
1,ETHUSDT,3575.691290,2.888505e+06,1.032840e+10
2,SOLUSDT,178.525097,2.627377e+07,4.690527e+09
3,DOGEUSDT,0.175816,1.166988e+10,2.051753e+09
4,ENAUSDT,1.028667,1.377274e+09,1.416756e+09
...,...,...,...,...
238,BTCDOMUSDT,2251.490323,2.243988e+03,5.052318e+06
239,AVAXUSDC,52.131526,8.285269e+04,4.319237e+06
240,DEFIUSDT,1287.625806,2.613028e+03,3.364602e+06
241,1000SHIBUSDC,0.028654,8.461280e+07,2.424465e+06


#1 최근 순위 2025-03-08 ~ 2025-04-07


['BTCUSDT',
 'ETHUSDT',
 'SOLUSDT',
 'XRPUSDT',
 'BTCUSDC',
 'DOGEUSDT',
 'ETHUSDC',
 '1000PEPEUSDT',
 'SUIUSDT',
 'BNBUSDT']

 데이터 수집 기간: 2024-03-08 00:00:00 ~ 2024-04-07 00:00:00
 ['BTCUSDT',
 'ETHUSDT',
 'SOLUSDT',
 'DOGEUSDT',
 'ENAUSDT', *
 '1000PEPEUSDT',
 'WIFUSDT', *
 'BOMEUSDT', *]


In [6]:
print(Future_db.get_top10_futures.__doc__)


시가총액 기준 상위 10개 선물 종목을 반환합니다.
        
        (self, symbols_option=1, symobls_list == None)

        Param

        interval = '30m'  : 시간간격 (1m, 3m, 5m, 15m, 30m, 1h, 2h, 4h, 6h, 8h, 12h, 1d, 3d, 1w, 1M)
        symbols_option==1 : api에서 받은 심볼을 슬라이싱 합니다.
        symbols_option==2 : 현재로부터 한달전의 마켓 켑을 두고, 약 422개를 계산한 후 10를 선별합니다.
        symbols_option==3 : 임의로 지정한 symbols_list 를 심볼로 사용합니다.(날짜를 변경할 때 사용)
            symobls_list: 심볼리스트를 지정해서 데이터를 가져옵니다.(option 3와 함꼐사용)
        


In [12]:
list(symbols_marketcap['symbol'][:10])


['BTCUSDT',
 'ETHUSDT',
 'SOLUSDT',
 'DOGEUSDT',
 'ENAUSDT',
 '1000PEPEUSDT',
 'WIFUSDT',
 'BOMEUSDT',
 'XRPUSDT',
 'BNBUSDT']

In [13]:
#2025년도 기준 심볼 적용
futures_10 = futures.get_top10_futures(symbols_option=3, symobls_list = list(symbols_marketcap['symbol'][:10]))
futures_10

[1/10] BTCUSDT 처리 중...
[2/10] ETHUSDT 처리 중...
[3/10] SOLUSDT 처리 중...
[4/10] DOGEUSDT 처리 중...
[5/10] ENAUSDT 처리 중...
[6/10] 1000PEPEUSDT 처리 중...
[7/10] WIFUSDT 처리 중...
[8/10] BOMEUSDT 처리 중...
[9/10] XRPUSDT 처리 중...
[10/10] BNBUSDT 처리 중...


,BTCUSDT,ETHUSDT,SOLUSDT,DOGEUSDT,ENAUSDT,1000PEPEUSDT,WIFUSDT,BOMEUSDT,XRPUSDT,BNBUSDT
timestamp,,,,,,,,,,
2019-09-08 17:30:00,10000.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-09-08 18:00:00,10000.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-09-08 18:30:00,10000.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-09-08 19:00:00,10354.62,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-09-08 19:30:00,10340.12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2025-04-07 22:00:00,79776.50,1580.12,109.00,0.15225,0.2945,0.006574,0.3492,0.001045,1.9392,559.32
2025-04-07 22:30:00,79919.50,1577.83,108.50,0.15103,0.2946,0.006543,0.3471,0.001039,1.9288,558.47
2025-04-07 23:00:00,79819.60,1565.44,107.76,0.14970,0.2927,0.006471,0.3456,0.001033,1.9089,556.59


In [ ]:
futures_10.to_parquet("futures_10_19_25.parquet", engine='pyarrow')
futures_10.to_excel('futures_10_19_25.xlsx',index = True )